In [30]:
from ctypes import *
import time
from sys import path
from os import sep
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dwf = cdll.LoadLibrary("libdwf.so")
constants_path = "/usr/share/digilent/waveforms/samples/py"
path.append(constants_path)
import dwfconstants as constants

In [31]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0)) #setting VDD = 3V
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.00V


In [32]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
WTIME_REG = 0x83
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3)) #Here, the freq of the communication is set.
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [33]:
write_register(ENABLE_REG, 0x01) #Power on / PON = 1
time.sleep(0.01)
write_register(ENABLE_REG, 0x0B) #Now, also RGBC enable / PON = 1, AEN = 1, WEN = 1
write_register(ATIME_REG, 0xF6) #To run for 24ms, upto a max count of 10,240
write_register(CONTROL_REG, 0x02) #Setting AGAIN=16
write_register(WTIME_REG, 0XFF) #Reseting the wtime register

True

In [34]:
#Function to test the sensor output
def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [35]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)

In [29]:
cleanup()
#call it everytime after work is done

In [38]:
print("--- Test 6.0: I2C Frequency Sweep (Min/Max Check) ---")

# 0Hz, 100Hz, 1kHz, 10kHz (Slow), 100kHz (Standard), 400kHz (Fast Mode Limit), 600kHz, 800kHz, 1MHz, 1.0e6, 1.2e6, 1.4e6, 1.6e6, 1.8e6, 2.0e6, 2.5e6 (Overclocking)
test_freq = [0, 100, 1000, 10e3, 50e3, 100e3, 200e3, 400e3, 500e3, 600e3, 800e3, 1000e3, 1.0e6, 1.2e6, 1.4e6, 1.6e6, 1.8e6, 2.0e6, 2.5e6]

print(f"{'Frequency':<15} | {'Status':<10} | {'ID Read'}")
print('\n'+"="*45)

pass_count = 0

for freq in test_freq:
    dwf.FDwfDigitalI2cRateSet(hdwf, c_double(freq))
    iNak = c_int()
    dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))
    time.sleep(0.05)

    val = read_register(0x92)

    if val == 0x44 or val == 0x4D:
        status = "PASS"
        pass_count += 1
        id_str = f"0x{val:02x}"
    elif val is None:
        status = "FAIL"
        id_str = f"NACK (No Response)"
    else:
        status = "FAIL"
        id_str = f"CORRUPT (Read 0x{val:02x})"

    if freq>=1000:
        print(f"{freq/1000:8.0f} kHz | {status:<10} | {id_str}")
    else: 
        print(f"{freq:9.0f} Hz | {status:<10} | {id_str}")

print("\n"+"="*45)

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
print("Reset I2C Clock to standard 100kHz")

--- Test 6.0: I2C Frequency Sweep (Min/Max Check) ---
Frequency       | Status     | ID Read

        0 Hz | FAIL       | NACK (No Response)
      100 Hz | FAIL       | NACK (No Response)
       1 kHz | PASS       | 0x4d
      10 kHz | PASS       | 0x4d
      50 kHz | PASS       | 0x4d
     100 kHz | PASS       | 0x4d
     200 kHz | PASS       | 0x4d
     400 kHz | PASS       | 0x4d
     500 kHz | PASS       | 0x4d
     600 kHz | PASS       | 0x4d
     800 kHz | PASS       | 0x4d
    1000 kHz | PASS       | 0x4d
    1000 kHz | PASS       | 0x4d
    1200 kHz | PASS       | 0x4d
    1400 kHz | PASS       | 0x4d
    1600 kHz | PASS       | 0x4d
    1800 kHz | PASS       | 0x4d
    2000 kHz | PASS       | 0x4d
    2500 kHz | PASS       | 0x4d

Reset I2C Clock to standard 100kHz
